In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import lightgbm as lgb
from sklearn.decomposition import TruncatedSVD
import lightgbm as lgb
import numpy as np
import pandas as pd
import os
import pickle
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir_path = '/content/drive/MyDrive/ML Project/data/'
df_train = pd.read_csv(data_dir_path + 'FINAL_TabularFeature_with_3TextEmbeddings_train.csv')
df_test = pd.read_csv(data_dir_path + 'FINAL_TabularFeature_with_3TextEmbeddings_test.csv')
df_train

,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,user_type,region_city,...,spacy_svd_30,spacy_svd_31,spacy_svd_32,spacy_svd_33,spacy_svd_34,spacy_svd_35,spacy_svd_36,spacy_svd_37,spacy_svd_38,spacy_svd_39
0,Омская область,Омск,Бытовая электроника,Настольные компьютеры,NaN,NaN,NaN,50.0,Private,Омская область_Омск,...,0.070859,-0.200402,0.399969,0.464117,-0.783324,0.042151,0.461139,0.033606,0.273473,0.345149
1,Башкортостан,Уфа,Хобби и отдых,Спорт и отдых,Зимние виды спорта,NaN,NaN,500.0,Private,Башкортостан_Уфа,...,-0.460100,0.109586,-0.204724,-0.232934,-0.253469,0.013493,0.305173,0.093155,0.241529,0.004352
2,Нижегородская область,Дзержинск,Для дома и дачи,Растения,NaN,NaN,NaN,80.0,Private,Нижегородская область_Дзержинск,...,-0.009834,-0.376687,0.270234,0.186656,-0.457154,0.029784,-0.187105,-0.593824,-0.213089,0.623080
3,Челябинская область,Миасс,Личные вещи,Детская одежда и обувь,Для девочек,Верхняя одежда,98-104 см (2-4 года),1500.0,Private,Челябинская область_Миасс,...,-0.146123,0.093856,-0.400256,-0.540356,0.629810,-0.108631,-0.443335,0.297444,0.112421,0.046438
4,Башкортостан,Уфа,Для дома и дачи,Бытовая техника,Для кухни,Плиты,NaN,18990.0,Shop,Башкортостан_Уфа,...,0.334175,0.183384,-0.140712,0.173028,-0.004446,0.184789,-0.004080,0.102282,0.066628,0.176290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202734,Ростовская область,Ростов-на-Дону,Услуги,Предложение услуг,"Ремонт, строительство",Сантехника,NaN,NaN,Private,Ростовская область_Ростов-на-Дону,...,0.480721,0.232176,-0.072481,-0.010870,0.239673,0.211701,-0.043929,0.193612,-0.128270,-0.304175
1202735,Тульская область,Тула,Недвижимость,Квартиры,Сдам,На длительный срок,2,13500.0,Company,Тульская область_Тула,...,0.241369,0.064511,0.057972,0.236461,0.251034,-0.197390,0.153372,0.239418,-0.094845,0.110294
1202736,Белгородская область,Алексеевка,Услуги,Предложение услуг,Другое,NaN,NaN,700.0,Private,Белгородская область_Алексеевка,...,-0.435914,0.283831,-0.043896,0.012694,0.239465,0.096773,0.053324,-0.124128,0.009042,-0.077851
1202737,Челябинская область,Челябинск,Личные вещи,Товары для детей и игрушки,Детские коляски,NaN,NaN,9000.0,Private,Челябинская область_Челябинск,...,0.093592,-0.526595,-0.191935,-0.310929,0.004045,-0.137020,-0.117629,-0.120800,0.071888,0.052071


In [ ]:
df_test

,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,user_type,region_city,...,spacy_svd_30,spacy_svd_31,spacy_svd_32,spacy_svd_33,spacy_svd_34,spacy_svd_35,spacy_svd_36,spacy_svd_37,spacy_svd_38,spacy_svd_39
0,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Аксессуары,NaN,NaN,50000.0,Private,Красноярский край_Красноярск,...,-0.322273,-0.001168,-0.379320,0.000552,-0.002261,0.064362,0.157960,-0.190988,-0.172207,0.084610
1,Ярославская область,Ярославль,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,21950.0,Company,Ярославская область_Ярославль,...,0.108131,0.112723,0.056348,0.078313,0.076306,-0.016968,0.022123,0.037728,0.052844,-0.016502
2,Тюменская область,Тюмень,Личные вещи,"Одежда, обувь, аксессуары",Мужская одежда,Пиджаки и костюмы,52–54 (XXL),2500.0,Private,Тюменская область_Тюмень,...,0.105742,0.015035,0.180098,-0.219161,-0.253191,-0.230806,-0.205053,0.281821,0.577967,0.178472
3,Новосибирская область,Новосибирск,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,32,750.0,Company,Новосибирская область_Новосибирск,...,0.007149,0.171786,0.181411,0.056841,-0.141842,-0.184129,0.066908,-0.142360,-0.032983,0.037495
4,Краснодарский край,Новороссийск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,42–44 (S),200.0,Private,Краснодарский край_Новороссийск,...,0.004518,-0.681913,-0.230038,-0.141053,-0.118111,0.202493,-0.027205,-0.149595,0.102804,-0.392306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300680,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Обувь,37,1000.0,Private,Красноярский край_Красноярск,...,-0.013230,-0.614357,-0.182608,-0.107221,-0.118657,-0.377602,0.230813,0.269826,-0.069822,0.159903
300681,Свердловская область,Екатеринбург,Бытовая электроника,Телефоны,LG,NaN,NaN,5500.0,Private,Свердловская область_Екатеринбург,...,0.037777,0.244492,-0.216132,0.093566,-0.036233,-0.127820,0.511694,-0.208028,-0.229156,0.217439
300682,Тульская область,Тула,Личные вещи,Товары для детей и игрушки,Товары для купания,NaN,NaN,150.0,Private,Тульская область_Тула,...,-0.100290,-0.262300,-0.080573,-0.250221,0.120220,-0.127277,0.253796,0.186099,0.152250,0.225946
300683,Свердловская область,Екатеринбург,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,24,300.0,Private,Свердловская область_Екатеринбург,...,0.666533,0.015940,-0.294694,-0.424865,0.389159,-0.019254,0.172634,0.147752,0.118220,0.071372


In [ ]:
df_train = df_train.drop(['item_id', 'user_id', 'item_seq_number', 'activation_date', 'image',
       'image_top_1'], axis=1)
df_test = df_test.drop(['item_id', 'user_id', 'item_seq_number', 'activation_date', 'image',
       'image_top_1'], axis=1)
df_test


,region,city,parent_category_name,category_name,param_1,param_2,param_3,price,user_type,region_city,...,spacy_svd_30,spacy_svd_31,spacy_svd_32,spacy_svd_33,spacy_svd_34,spacy_svd_35,spacy_svd_36,spacy_svd_37,spacy_svd_38,spacy_svd_39
0,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Аксессуары,NaN,NaN,50000.0,Private,Красноярский край_Красноярск,...,-0.322273,-0.001168,-0.379320,0.000552,-0.002261,0.064362,0.157960,-0.190988,-0.172207,0.084610
1,Ярославская область,Ярославль,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,21950.0,Company,Ярославская область_Ярославль,...,0.108131,0.112723,0.056348,0.078313,0.076306,-0.016968,0.022123,0.037728,0.052844,-0.016502
2,Тюменская область,Тюмень,Личные вещи,"Одежда, обувь, аксессуары",Мужская одежда,Пиджаки и костюмы,52–54 (XXL),2500.0,Private,Тюменская область_Тюмень,...,0.105742,0.015035,0.180098,-0.219161,-0.253191,-0.230806,-0.205053,0.281821,0.577967,0.178472
3,Новосибирская область,Новосибирск,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,32,750.0,Company,Новосибирская область_Новосибирск,...,0.007149,0.171786,0.181411,0.056841,-0.141842,-0.184129,0.066908,-0.142360,-0.032983,0.037495
4,Краснодарский край,Новороссийск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Платья и юбки,42–44 (S),200.0,Private,Краснодарский край_Новороссийск,...,0.004518,-0.681913,-0.230038,-0.141053,-0.118111,0.202493,-0.027205,-0.149595,0.102804,-0.392306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300680,Красноярский край,Красноярск,Личные вещи,"Одежда, обувь, аксессуары",Женская одежда,Обувь,37,1000.0,Private,Красноярский край_Красноярск,...,-0.013230,-0.614357,-0.182608,-0.107221,-0.118657,-0.377602,0.230813,0.269826,-0.069822,0.159903
300681,Свердловская область,Екатеринбург,Бытовая электроника,Телефоны,LG,NaN,NaN,5500.0,Private,Свердловская область_Екатеринбург,...,0.037777,0.244492,-0.216132,0.093566,-0.036233,-0.127820,0.511694,-0.208028,-0.229156,0.217439
300682,Тульская область,Тула,Личные вещи,Товары для детей и игрушки,Товары для купания,NaN,NaN,150.0,Private,Тульская область_Тула,...,-0.100290,-0.262300,-0.080573,-0.250221,0.120220,-0.127277,0.253796,0.186099,0.152250,0.225946
300683,Свердловская область,Екатеринбург,Личные вещи,Детская одежда и обувь,Для девочек,Обувь,24,300.0,Private,Свердловская область_Екатеринбург,...,0.666533,0.015940,-0.294694,-0.424865,0.389159,-0.019254,0.172634,0.147752,0.118220,0.071372


In [ ]:
y_test = df_test['deal_probability']

# Stack

In [ ]:
params = {
            'objective': 'regression',
            'metric': 'rmse',
            'boosting_type': 'gbdt',
            'learning_rate': 0.02,
            'num_leaves': 512,
            'max_depth': 20,
            'bagging_seed': 42,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'feature_fraction': 0.8,
            'lambda_l1': 2.25,
            'lambda_l2': 0.015,
            'verbose': -1
            }

In [ ]:
model_dir_path = '/content/drive/MyDrive/ML Project/code/Stackingmodels'

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
train_meta_features = np.zeros((df_train.shape[0], 3))
test_meta_features = np.zeros((df_test.shape[0], 3))
train_meta_features.shape

(1202739, 3)

In [ ]:
embedding_sets = [
    [col for col in df_train.columns if col.startswith("tfidf_svd")],  # Embedding 1
    [col for col in df_train.columns if col.startswith("fasttext_svd")],  # Embedding 2
    [col for col in df_train.columns if col.startswith("spacy_svd")]  # Embedding 3
]

In [ ]:
df_train.columns

Index(['region', 'city', 'parent_category_name', 'category_name', 'param_1',
       'param_2', 'param_3', 'price', 'user_type', 'region_city',
       ...
       'spacy_svd_30', 'spacy_svd_31', 'spacy_svd_32', 'spacy_svd_33',
       'spacy_svd_34', 'spacy_svd_35', 'spacy_svd_36', 'spacy_svd_37',
       'spacy_svd_38', 'spacy_svd_39'],
      dtype='object', length=121)

In [ ]:
y_train = df_train['deal_probability']

# First Stage Models * 3


In [ ]:
# 开始一级模型训练
for i, embedding_cols in enumerate(embedding_sets):
    print(f"Training first-stage model {i+1} with embeddings: {embedding_cols[0]} to {embedding_cols[-1]}")

    X_train = df_train[embedding_cols].values
    X_test = df_test[embedding_cols].values

    # 存储测试集预测值（每折平均）
    test_fold_predictions = np.zeros(X_test.shape[0])

    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        # 创建 LightGBM 数据集
        train_data = lgb.Dataset(X_tr, label=y_tr)
        val_data = lgb.Dataset(X_val, label=y_val, free_raw_data=False)

        # 训练 LightGBM 模型
        callbacks = [
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=20)
        ]
        model = lgb.train(
            params,
            train_data,
            num_boost_round=1000,
            valid_sets=[train_data, val_data],
            callbacks=callbacks

        )

        # 保存每个模型
        model_filename = f'{model_dir_path}/first_stage_model_{i+1}_fold_{fold+1}.txt'
        model.save_model(model_filename)
        print(f"Model {i+1} Fold {fold+1} saved to {model_filename}")

        # 验证集预测
        train_meta_features[val_idx, i] = model.predict(X_val, num_iteration=model.best_iteration)

        # 测试集预测
        test_fold_predictions += model.predict(X_test, num_iteration=model.best_iteration) / kf.n_splits

    # 保存每个一级模型对测试集的预测
    test_meta_features[:, i] = test_fold_predictions

# 查看一级模型生成的 Meta Features
print("Meta-features shape (train):", train_meta_features.shape)
print("Meta-features shape (test):", test_meta_features.shape)

Training first-stage model 1 with embeddings: tfidf_svd_0 to tfidf_svd_4
Training until validation scores don't improve for 50 rounds
[20]	training's rmse: 0.254214	valid_1's rmse: 0.255801
[40]	training's rmse: 0.251212	valid_1's rmse: 0.253063
[60]	training's rmse: 0.2495	valid_1's rmse: 0.251607
[80]	training's rmse: 0.248421	valid_1's rmse: 0.250786
[100]	training's rmse: 0.247703	valid_1's rmse: 0.250308
[120]	training's rmse: 0.247172	valid_1's rmse: 0.250021
[140]	training's rmse: 0.246743	valid_1's rmse: 0.249844
[160]	training's rmse: 0.246385	valid_1's rmse: 0.249725
[180]	training's rmse: 0.246066	valid_1's rmse: 0.249644
[200]	training's rmse: 0.245758	valid_1's rmse: 0.249581
[220]	training's rmse: 0.245484	valid_1's rmse: 0.24953
[240]	training's rmse: 0.245224	valid_1's rmse: 0.249494
[260]	training's rmse: 0.244981	valid_1's rmse: 0.249466
[280]	training's rmse: 0.244747	valid_1's rmse: 0.249444
[300]	training's rmse: 0.244532	valid_1's rmse: 0.24943
[320]	training's rm

In [ ]:
# 将 Meta Features 和 Tabular Features 合并，准备元模型训练
tabular_features = [col for col in df_train.columns if not col.startswith(("tfidf_svd", "fasttext_svd", "spacy_svd")) and col != 'deal_probability']


In [ ]:
# 合并一级模型的输出和 Tabular Features
X_meta_train = np.hstack([train_meta_features, df_train[tabular_features].values])
X_meta_test = np.hstack([test_meta_features, df_test[tabular_features].values])


In [ ]:
X_meta_train

array([[0.197765253100973, 0.20141205927235234, 0.2370829397091052, ...,
        0.0038880740542951, 3.3333333333333335e-05, 0],
       [0.15741392949261376, 0.07740918893082918, 0.06248983434178254,
        ..., 0.0446370675811513, 7.142295962460093e-06, 0],
       [0.08868171554202743, 0.10776285919224125, 0.17357745908391434,
        ..., 0.0343541927863076, 7.9960019990005e-05, 0],
       ...,
       [0.2310229106570014, 0.3229063582052782, 0.2642133801771755, ...,
        0.0451741650074788, 7e-06, 0],
       [0.35017239367998176, 0.2632484582259241, 0.24280115142335543,
        ..., 2.100934689605189, 0.0002571281641049, 0],
       [0.10189059056461729, 0.015048289880422349, 0.06422463373045327,
        ..., 0.0317288434882244, 8.316688866681482e-07, 1]], dtype=object)

In [ ]:
df_train_copy = pd.read_csv(data_dir_path + 'FINAL_TabularFeature_with_3TextEmbeddings_train.csv')
df_test_copy = pd.read_csv(data_dir_path + 'FINAL_TabularFeature_with_3TextEmbeddings_test.csv')

In [ ]:

train_meta_features

array([[0.19776525, 0.20141206, 0.23708294],
       [0.15741393, 0.07740919, 0.06248983],
       [0.08868172, 0.10776286, 0.17357746],
       ...,
       [0.23102291, 0.32290636, 0.26421338],
       [0.35017239, 0.26324846, 0.24280115],
       [0.10189059, 0.01504829, 0.06422463]])

In [ ]:
train_meta_df = pd.DataFrame(
    train_meta_features,
    columns=['tfidf_svd5_prob', 'fasttext_svd42_prob', 'spacy_svd40_prob']
)

item_user_df = df_train_copy[['item_id', 'user_id']].reset_index(drop=True)

train_meta_features_with_ids = pd.concat([item_user_df, train_meta_df], axis=1)
train_meta_features_with_ids

,item_id,user_id,tfidf_svd5_prob,fasttext_svd42_prob,spacy_svd40_prob
0,771fda0ff8ff,969f1c4acd66,0.197765,0.201412,0.237083
1,3c296fc97db9,cf345facfe6b,0.157414,0.077409,0.062490
2,13ecb01f22c1,a4ad6dcc8446,0.088682,0.107763,0.173577
3,f3a81005ce3d,a41d7a0cd76f,0.125188,0.113663,0.215299
4,bc91d5eb9229,19757fb0047c,0.099597,0.071398,0.080358
...,...,...,...,...,...
1202734,f09ae5daaf01,f9b5910f655a,0.235149,0.424500,0.320925
1202735,6eab9f61133f,1decc320fb12,0.114521,0.312441,0.131626
1202736,9b5c1f4fca75,196588f9b315,0.231023,0.322906,0.264213
1202737,22c8381d9eab,39b04d8ac30a,0.350172,0.263248,0.242801


In [ ]:
test_meta_df = pd.DataFrame(
    test_meta_features,
    columns=['tfidf_svd5_prob', 'fasttext_svd42_prob', 'spacy_svd40_prob']
)

item_user_df_test = df_test_copy[['item_id', 'user_id']].reset_index(drop=True)

test_meta_features_with_ids = pd.concat([item_user_df_test, test_meta_df], axis=1)
test_meta_features_with_ids

,item_id,user_id,tfidf_svd5_prob,fasttext_svd42_prob,spacy_svd40_prob
0,bd45a493569d,6be13830198e,0.145480,0.105418,0.125192
1,321127a6a33b,98b93e9aaf33,0.215047,0.127215,0.128890
2,d976d5211526,7246f2b892e9,0.138693,0.083785,0.134336
3,c046d8685345,3ceed1f36bec,0.128141,0.094044,0.142646
4,98a8ba1ff4c4,feb28eeb11a9,0.075082,0.022229,0.027074
...,...,...,...,...,...
300680,07a88b2f9ffc,868bce3fbbb2,0.084698,0.073209,0.102568
300681,a7b7029023e4,3217919a362d,0.337099,0.193074,0.190216
300682,efecf8d66b20,cdb3b17927a8,0.191250,0.091205,0.082258
300683,b4e141ae7703,845b032118f6,0.062663,0.045960,0.049148


In [ ]:
first_stage_pred_data_dir = '/content/drive/MyDrive/ML Project/data/FirstStageModel_predicted/'
train_meta_features_with_ids.to_csv(first_stage_pred_data_dir + 'train_firststage_3embeddings_prob.csv', index=False)
test_meta_features_with_ids.to_csv(first_stage_pred_data_dir + 'test_firststage_3embeddings_prob.csv', index=False)

In [ ]:

meta_feature_columns = ['tfidf_svd5_prob', 'fasttext_svd42_prob', 'spacy_svd40_prob']

meta_train_df = pd.DataFrame(X_meta_train, columns=meta_feature_columns + tabular_features)

meta_train_df.columns


Index(['tfidf_svd5_prob', 'fasttext_svd42_prob', 'spacy_svd40_prob', 'region',
       'city', 'parent_category_name', 'category_name', 'param_1', 'param_2',
       'param_3', 'price', 'user_type', 'region_city', 'all_category',
       'category_param_1', 'region_category_user', 'city_category_user',
       'category_price_mean', 'category_price_std', 'category_price_skew',
       'city_price_mean', 'city_price_max', 'city_price_skew', 'title_length',
       'description_length', 'title_word_count', 'description_word_count',
       'title_has_keyword', 'description_has_keyword', 'title_digit_count',
       'description_digit_count', 'description_newline_count', 'price_log',
       'price_bin', 'price_to_category_mean', 'price_to_category_max',
       'description_missing'],
      dtype='object')

In [ ]:

meta_feature_columns = ['tfidf_svd5_prob', 'fasttext_svd42_prob', 'spacy_svd40_prob']

meta_test_df = pd.DataFrame(X_meta_test, columns=meta_feature_columns + tabular_features)

meta_test_df.columns


Index(['tfidf_svd5_prob', 'fasttext_svd42_prob', 'spacy_svd40_prob', 'region',
       'city', 'parent_category_name', 'category_name', 'param_1', 'param_2',
       'param_3', 'price', 'user_type', 'region_city', 'all_category',
       'category_param_1', 'region_category_user', 'city_category_user',
       'category_price_mean', 'category_price_std', 'category_price_skew',
       'city_price_mean', 'city_price_max', 'city_price_skew', 'title_length',
       'description_length', 'title_word_count', 'description_word_count',
       'title_has_keyword', 'description_has_keyword', 'title_digit_count',
       'description_digit_count', 'description_newline_count', 'price_log',
       'price_bin', 'price_to_category_mean', 'price_to_category_max',
       'description_missing'],
      dtype='object')

In [ ]:
categorical_features = [
    'region', 'city', 'parent_category_name',
    'category_name', 'param_1', 'param_2','param_3', 'user_type',
    'region_city', 'all_category','category_param_1', 'region_category_user', 'city_category_user',
    'title_has_keyword', 'description_has_keyword',
    'price_bin', 'description_missing'
]


In [ ]:
def fit_lgb(
        train_X, train_y, test_X, test_y, categorical_features,
        params=None,
        num_boost_round=1000,
        early_stopping_rounds=50,
        eval_period=20,
    ):

    if params is None:
        params = {
            'objective': 'regression',
            'metric': 'rmse',
            'boosting_type': 'gbdt',
            'learning_rate': 0.02,
            'num_leaves': 512,
            'max_depth': 20,

            'bagging_seed': 42,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'feature_fraction': 0.8,
            'lambda_l1': 2.25,
            'lambda_l2': 0.015,

            'verbose': -1
        }

    # label encoding for cat var
    for col in categorical_features:
        train_X[col] = train_X[col].astype('category')
        test_X[col] = test_X[col].astype('category')

    # validation set for early stopping
    train_X_split, val_X_split, train_y_split, val_y_split = train_test_split(
        train_X, train_y, test_size=0.2, random_state=42, shuffle=True
    )
    lgb_train = lgb.Dataset(
        train_X_split, label=train_y_split, categorical_feature=categorical_features
    )
    lgb_val = lgb.Dataset(
        val_X_split, label=val_y_split, categorical_feature=categorical_features,
        reference=lgb_train
    )

    callbacks = [
        lgb.early_stopping(stopping_rounds=early_stopping_rounds),
        lgb.log_evaluation(period=eval_period)
    ]

    model = lgb.train(
        params,
        lgb_train,
        num_boost_round=num_boost_round,
        valid_sets=[lgb_train, lgb_val],
        callbacks=callbacks
    )

    preds = model.predict(test_X, num_iteration=model.best_iteration)
    rmse = mean_squared_error(test_y, preds, squared=False)
    print(f"RMSE: {rmse}")

    return model

In [ ]:
meta_train_df

,tfidf_svd5_prob,fasttext_svd42_prob,spacy_svd40_prob,region,city,parent_category_name,category_name,param_1,param_2,param_3,...,title_has_keyword,description_has_keyword,title_digit_count,description_digit_count,description_newline_count,price_log,price_bin,price_to_category_mean,price_to_category_max,description_missing
0,0.197765,0.201412,0.237083,Омская область,Омск,Бытовая электроника,Настольные компьютеры,NaN,NaN,NaN,...,True,True,0,0,0,3.931826,0,0.003888,0.000033,0
1,0.157414,0.077409,0.06249,Башкортостан,Уфа,Хобби и отдых,Спорт и отдых,Зимние виды спорта,NaN,NaN,...,True,True,4,4,0,6.216606,1,0.044637,0.000007,0
2,0.088682,0.107763,0.173577,Нижегородская область,Дзержинск,Для дома и дачи,Растения,NaN,NaN,NaN,...,False,False,0,15,11,4.394449,0,0.034354,0.00008,0
3,0.125188,0.113663,0.215299,Челябинская область,Миасс,Личные вещи,Детская одежда и обувь,Для девочек,Верхняя одежда,98-104 см (2-4 года),...,False,True,2,2,0,7.313887,2,0.190373,0.000005,0
4,0.099597,0.071398,0.080358,Башкортостан,Уфа,Для дома и дачи,Бытовая техника,Для кухни,Плиты,NaN,...,False,False,0,47,90,9.85172,4,2.354832,0.003797,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202734,0.235149,0.4245,0.320925,Ростовская область,Ростов-на-Дону,Услуги,Предложение услуг,"Ремонт, строительство",Сантехника,NaN,...,False,False,0,0,0,NaN,-1,NaN,NaN,0
1202735,0.114521,0.312441,0.131626,Тульская область,Тула,Недвижимость,Квартиры,Сдам,На длительный срок,2,...,False,False,6,6,0,9.510519,4,0.006762,0.000084,0
1202736,0.231023,0.322906,0.264213,Белгородская область,Алексеевка,Услуги,Предложение услуг,Другое,NaN,NaN,...,False,False,0,4,0,6.552508,1,0.045174,0.000007,0
1202737,0.350172,0.263248,0.242801,Челябинская область,Челябинск,Личные вещи,Товары для детей и игрушки,Детские коляски,NaN,NaN,...,False,True,0,2,0,9.105091,3,2.100935,0.000257,0


# Train the Meta Model


In [ ]:
meta_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202739 entries, 0 to 1202738
Data columns (total 37 columns):
 #   Column                     Non-Null Count    Dtype   
---  ------                     --------------    -----   
 0   tfidf_svd5_prob            1202739 non-null  object  
 1   fasttext_svd42_prob        1202739 non-null  object  
 2   spacy_svd40_prob           1202739 non-null  object  
 3   region                     1202739 non-null  category
 4   city                       1202739 non-null  category
 5   parent_category_name       1202739 non-null  category
 6   category_name              1202739 non-null  category
 7   param_1                    1153263 non-null  category
 8   param_2                    679119 non-null   category
 9   param_3                    512840 non-null   category
 10  price                      1134146 non-null  object  
 11  user_type                  1202739 non-null  category
 12  region_city                1202739 non-null  category
 1

In [ ]:
# 将所有 object 类型的列转换为数值型
for col in meta_train_df.select_dtypes(include='object').columns:
    try:
        # 尝试直接转换为数值型
        meta_train_df[col] = pd.to_numeric(meta_train_df[col], errors='coerce')
    except Exception as e:
        print(f"Column {col} could not be directly converted: {e}")
print(meta_train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202739 entries, 0 to 1202738
Data columns (total 37 columns):
 #   Column                     Non-Null Count    Dtype   
---  ------                     --------------    -----   
 0   tfidf_svd5_prob            1202739 non-null  float64 
 1   fasttext_svd42_prob        1202739 non-null  float64 
 2   spacy_svd40_prob           1202739 non-null  float64 
 3   region                     1202739 non-null  category
 4   city                       1202739 non-null  category
 5   parent_category_name       1202739 non-null  category
 6   category_name              1202739 non-null  category
 7   param_1                    1153263 non-null  category
 8   param_2                    679119 non-null   category
 9   param_3                    512840 non-null   category
 10  price                      1134146 non-null  float64 
 11  user_type                  1202739 non-null  category
 12  region_city                1202739 non-null  category
 1

In [ ]:
# 将所有 object 类型的列转换为数值型
for col in meta_test_df.select_dtypes(include='object').columns:
    try:
        # 尝试直接转换为数值型
        meta_test_df[col] = pd.to_numeric(meta_test_df[col], errors='coerce')
    except Exception as e:
        print(f"Column {col} could not be directly converted: {e}")

In [ ]:
meta_model = fit_lgb(meta_train_df, y_train, meta_test_df, y_test, categorical_features)


Training until validation scores don't improve for 50 rounds
[20]	training's rmse: 0.241407	valid_1's rmse: 0.244527
[40]	training's rmse: 0.23129	valid_1's rmse: 0.236299
[60]	training's rmse: 0.225383	valid_1's rmse: 0.232207
[80]	training's rmse: 0.221689	valid_1's rmse: 0.230121
[100]	training's rmse: 0.219187	valid_1's rmse: 0.229059
[120]	training's rmse: 0.217247	valid_1's rmse: 0.228454
[140]	training's rmse: 0.215874	valid_1's rmse: 0.228146
[160]	training's rmse: 0.21475	valid_1's rmse: 0.227962
[180]	training's rmse: 0.213883	valid_1's rmse: 0.227862
[200]	training's rmse: 0.213162	valid_1's rmse: 0.22781
[220]	training's rmse: 0.212481	valid_1's rmse: 0.227728
[240]	training's rmse: 0.211886	valid_1's rmse: 0.227655
[260]	training's rmse: 0.211356	valid_1's rmse: 0.227593
[280]	training's rmse: 0.210947	valid_1's rmse: 0.22756
[300]	training's rmse: 0.210636	valid_1's rmse: 0.227556
[320]	training's rmse: 0.210291	valid_1's rmse: 0.227545
[340]	training's rmse: 0.209994	val

In [ ]:
model_dir_path_meta = '/content/drive/MyDrive/ML Project/code/Stackingmodels/meta/'

In [ ]:
# booster = meta_model.booster_

meta_model_filename = f'{model_dir_path_meta}/meta_model_1.txt'
meta_model.save_model(meta_model_filename)
print(f"Meta-model saved to {meta_model_filename}")

Meta-model saved to /content/drive/MyDrive/ML Project/code/Stackingmodels/meta//meta_model_1.txt
